# 과제: LLM을 활용한 일상 문제 해결 앱 프로토타입 제작
## 주제: 그린라이트 감지기 (Love Signal Detector)

---

### 1. 문제 발굴 및 정의 (Problem Definition)

#### 1.1 발견한 불편함
관심 있는 이성(썸남/썸녀)에게 카톡이 왔을 때, 이게 단순히 예의상 보낸 답장인지 아니면 나에게 호감이 있어서 보낸 건지 헷갈린다.
친구들에게 캡처해서 물어보기도 지치고, 밤새 의미 부여하느라 잠을 설친다.

#### 1.2 문제 정의
- **Target:** 썸을 타고 있거나 짝사랑 중인 2030 남녀
- **Problem Statement:** "상대방과의 카톡 대화 내용을 입력하면, 말투와 반응을 분석해 객관적인 호감도 확률(%)을 알려주고, 센스 있는 답장까지 추천해 주는 도구가 필요하다."
- **App Name:** **'그린라이트 감지기' (Love Signal Detector)**

In [4]:
# 필수 라이브러리 임포트
import os
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일 로드 (로컬 환경용)
load_dotenv()

# API Key 설정
api_key = os.getenv("OPENAI_API_KEY")

# 키가 없을 경우 (Colab 등에서 수동 입력 시 사용)
if not api_key:
    # Colab 사용 시 아래 주석을 풀고 userdata 사용 가능
    # from google.colab import userdata
    # api_key = userdata.get('OPENAI_API_KEY')
    api_key = "sk-..." # 혹은 여기에 직접 입력 (보안 주의!)

client = OpenAI(api_key=api_key)
print("OpenAI Client 설정 완료!")

OpenAI Client 설정 완료!


### 2. 프롬프트 엔지니어링 (Prompt Engineering)

#### 2.1 1차 프로토타입 (V1)
기본적인 호감도 분석 기능을 구현합니다.
- **Role:** 연애 심리 분석 전문가
- **Output:** 호감도 확률, 이유, 추천 답장

In [5]:
def get_love_signal_analysis_v1(chat_log):
    system_prompt = """
    너는 연애 심리 분석 전문가이자 100만 유튜버 연애 코치야.
    사용자가 이성과의 카톡 대화 로그를 입력하면, 상대방의 호감도를 분석하고 조언해줘.

    [분석 기준]
    - 질문이 계속 이어지는가? (호감 신호)
    - 단답형인가? (부정 신호)
    - 이모티콘이나 맞춤법, 답장 길이는 어떠한가?

    [출력 형식]
    1. 💚 호감도 확률: OO%
    2. 🕵️ 코치의 분석:
       - (이유 1)
       - (이유 2)
       - (이유 3)
    3. 💌 추천 답장:
       - (1번 버전)
       - (2번 버전)
       - (3번 버전)
    """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": chat_log}
        ],
        temperature=0.7
    )
    return completion.choices[0].message.content

### 3. 테스트 및 개선 (Testing & Iteration)

#### 테스트 데이터 (긍정 시그널)

In [6]:
chat_log_good = """
나: 오늘 날씨 진짜 춥다 ㅠㅠ 너 따뜻하게 입고 나갔어?
상대방: 웅웅 패딩 입었지! ㅋㅋ 너는? 감기 걸리면 안 돼 ㅠㅠ
나: 나도 롱패딩 입었어 ㅋㅋ 오늘 점심 뭐 먹어?
상대방: 오늘 팀장님이랑 김치찌개 먹으러 갈 듯.. 너는 뭐 먹는데? 맛있는 거 먹어!
"""

print(get_love_signal_analysis_v1(chat_log_good))

1. 💚 호감도 확률: 85%
2. 🕵️ 코치의 분석:
   - 상대방이 질문에 답하면서도 "너는?"이라고 되물으며 대화를 이어가려는 의도가 보입니다. 이는 호감 신호입니다.
   - 이모티콘과 "ㅋㅋ" 같은 표현이 자연스럽게 사용되어 긍정적인 분위기를 조성하고 있습니다.
   - 답장 길이도 적당하고, 맞춤법에도 신경을 쓴 모습이 보여져 상대방이 대화에 신경을 쓰고 있다는 것을 알 수 있습니다.
3. 💌 추천 답장:
   - "김치찌개 맛있겠다! 나도 국물 있는 음식 먹고 싶네 ㅋㅋ 나도 맛있는 점심 먹을게~"
   - "팀장님이랑 좋은 시간 보내! 나도 맛있는 거 먹고 올게 ㅎㅎ"
   - "오늘은 따뜻한 음식이 진리지! 나도 찌개 같은 거 먹을까 생각 중이야 ㅋㅋ 맛있게 먹어~"


#### 3.2 프롬프트 개선 (V2)
초기 모델 피드백: "너무 딱딱하다", "더 재미있었으면 좋겠다"
-> **페르소나 강화 (MBTI 과몰입러) 및 말투 변경**

In [ ]:
def get_love_signal_analysis_v2(chat_log):
    # 개선된 프롬프트 (Ver.2)
    system_prompt_v2 = """
    너는 20대 연애 상담 분야 1위 유튜버이자, MBTI 과몰입러야.
    사용자의 카톡 대화를 보고, 친구처럼 솔직하고 시원하게 조언해줘야 해.

    [필수 분석 항목]
    1. 🚦 **그린라이트 지수**: 0~100% (이유를 팩트 폭격으로 설명)
    2. 🧠 **상대방 예상 심리**: (MBTI 관점에서 추측, 예: T라서 그런지, F라서 그런지 등)
    3. 🚀 **추천 답장**: 상대방의 호기심을 유발할 수 있는 센스 있는 답장 2개 추천.

    [제약 조건]
    - 말투는 "야," "솔직히 말해서," "~인 듯" 같은 구어체를 사용해.
    - 상대방이 어장관리 중이라면 따끔하게 정신 차리라고 말해줘.
    """

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt_v2},
            {"role": "user", "content": chat_log}
        ],
        temperature=0.8 # 창의성 높임
    )
    return completion.choices[0].message.content

# 개선된 버전 테스트 실행
print(get_love_signal_analysis_v2(chat_log_good))

### 4. 앱 구현 계획 (Implementation)
- **Framework:** Streamlit
- **File:** `app.py`
- 위에서 검증된 V2 프롬프트를 사용하여 웹 애플리케이션으로 배포합니다.